In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import random
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import *
import operator
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score


In [42]:
def preprocess_text(df):

    st1 =[]
    st2_flag = []
    st3 = []
    for idx, row in df.iterrows():
            #filling df1['mailid'] column
            strg = row[0]
            words = strg.split(None, 1)[0]
            st1.append(words)
            #filling df1['spam_or_ham'] column
            words2 = strg.split(None, 1)[1]
            st2 = words2.split(None, 1)[0]
            if st2 =='ham':
                st2_flag.append(0)
            else:
                st2_flag.append(1)
            #Filling df1['content'] column 
            d = ' '
            words3 = words2.split(None, 1)[1]
            st3.append(words3)

    df1 = pd.DataFrame()
    df1['mailid']=st1
    df1['spam_or_ham']=st2_flag
    df1['content']=st3
    
    #Filling df1['spam_word_count'] with zeros for now
    zeros = []
    for i in range(0,len(df1['mailid'])):
        zeros.append(0)
    df1['spam_word_count'] = zeros
    print df1.head(10)
    return df1

In [56]:
def spam_list_creation(df1):
    notpos = ['PRP','IN','DT','WDT','WP','WRB','TO','MD','EX']
    stop_words = set(stopwords.words("english"))
    #put words in ham spam list and put their wrd count too
    dict_spam = {}
    dict_ham = {}
    for idx, row in df1.iterrows():
        if row['spam_or_ham']==1:
            wrds = row['content'].split(' ')
            for w, c in zip(*[iter(wrds)]*2):
                if not w.isdigit():
                    if w not in stop_words:
                        tagged = nltk.pos_tag([w])
                        #if (filter(lambda word_tag: word_tag[1] in notpos, tagged)):
                        #continue
                        if w in dict_spam.keys():
                            dict_spam[w]=dict_spam[w]+c
                        else:
                            dict_spam[w]=c
        else:
            wrds = row['content'].split(' ')
            for w, c in zip(*[iter(wrds)]*2):
                if not w.isdigit():
                    if w not in stop_words:
                        tagged = nltk.pos_tag([w])
                        #if (filter(lambda word_tag: word_tag[1] in notpos, tagged)):
                        #continue
    
                        if w in dict_ham.keys():
                            dict_ham[w]=dict_ham[w]+c
                        else:
                            dict_ham[w]=c
                break
    ham_list = sorted(dict_ham.keys(), key=dict_ham.get)[-70:]
    spam_list = sorted(dict_spam.keys(), key=dict_spam.get)[-70:]
    return spam_list

In [55]:
def check_for_spam_words(spam_list, df1):
    #filling df1['spam_word_count'] column 
    spam_word_cnt_list = []
    for idx, row in df1.iterrows():
        spam_word_count = 0
        wrds = row['content'].split(' ')
        for w, c in zip(*[iter(wrds)]*2):
            if not w.isdigit():
                if w in spam_list:
                    spam_word_count=spam_word_count+int(c)
        spam_word_cnt_list.append(spam_word_count)
        
    df1['spam_word_count']=spam_word_cnt_list
    print df1.head(10)
    return df1

In [61]:
def classifier():
    #csv file containing preprocessed training dataset
    df_train = pd.read_csv("processed_trainingdata2.csv")
    y_train =pd.DataFrame()
    y_train =df_train['spam_or_ham']
    X_train =df_train
    X_train.drop('spam_or_ham', 1, inplace=True)
    X_train.drop('mailid', 1, inplace=True)
    X_train.drop('content', 1, inplace=True)

    #Initialise the classification model
    nb = MultinomialNB()
    
    print("Fit the training dataset")
    nb.fit(X_train, y_train)
    
    #csv file containing preprocessed testing dataset
    df_test = pd.read_csv("processed_testingdata2.csv")
    y_test =pd.DataFrame()
    y_test =df_test['spam_or_ham']
    X_test =df_test
    X_test.drop('spam_or_ham', 1, inplace=True)
    X_test.drop('mailid', 1, inplace=True)
    X_test.drop('content', 1, inplace=True)
    
    print("predicting in the testing dataset")
    prediction = nb.predict(X_test)
    
    #Evaluate Accuracy
    print_results(y_test, prediction)
    
    prediction_list = []
    for each in prediction:
        if each==1:
            prediction_list.append("spam")
        else:
            prediction_list.append("ham")
            
    df_test = pd.read_csv("processed_testingdata2.csv")
    df_op =pd.DataFrame()
    df_op['ID']=df_test['mailid']
    df_op['spam/ham']=prediction_list
    filename = 'output_file.csv'
    df_op.to_csv(filename, index=False)
    return

In [48]:
def print_results(y_test, prediction):
    print("Accuracy score:", accuracy_score(y_test, prediction)*100)
    print("R^2:", r2_score(y_test,prediction))
    rmse = np.sqrt(mean_squared_error(y_test, prediction))
    print("Mean squared error:", mean_squared_error(y_test, prediction))
    print("Root mean squared error: {}".format(rmse))
    print("Mean absolute error:", mean_absolute_error(y_test,prediction))

In [52]:
def main():
    print("preprocess training data")
    df = pd.read_csv("data\set_train")
    df1 = preprocess_text(df)
    filename = 'preprocessing1.csv'
    df1.to_csv(filename, index=False)
    
    print("Spam words list creation")
    spam_list = spam_list_creation(df1)
    sdf = pd.read_csv("spam_wrds.csv")
    spamlist = [[sdf] + spam_list]
    
    print("Checking for Spam words")
    df_train = pd.DataFrame()
    df_train = check_for_spam_words(spam_list, df1)
    filename = 'processed_trainingdata2.csv'
    df_train.to_csv(filename, index=False)
        
    print("preprocess testing data")
    df2 = pd.read_csv("data\set_test")
    df3=preprocess_text(df2)
    filename = 'preprocessing_test1.csv'
    df3.to_csv(filename, index=False)

    #Checking for Spam words
    df_test = pd.DataFrame()
    df_test = check_for_spam_words(spam_list, df3)
    filename = 'processed_testingdata2.csv'
    df_test.to_csv(filename, index=False)
    
    print("Classifying emails")
    classifier()

In [ ]:
if __name__ == '__main__':
    main()

preprocess training data
     mailid  spam_or_ham                                            content  \
0  /000/031            0  don 1 kim 5 dave 1 39 1 customer 1 38 2 thanks...   
1  /000/044            0  to 1 mimeole 1 for 1 kim 3 110 1 smtpsvc 1 39 ...   
2  /000/057            0  to 2 mimeole 1 110 1 smtpsvc 1 39 1 type 1 has...   
3  /000/069            0  smtp 1 for 3 visit 1 its 1 0700 4 of 5 on 1 in...   
4  /000/076            0  for 4 33 2 are 1 file 2 40 1 thread 2 exchange...   
5  /000/077            1  for 2 mailer 1 center 1 exchange 1 transfer 1 ...   
6  /000/079            1  smtp 1 for 2 mailer 1 info 2 center 1 exchange...   
7  /000/083            1  need 1 else 1 iso 1 200 3 charset 3 windows 1 ...   
8  /000/113            1  for 2 34 1 39 2 mailer 1 0700 1 43 3 42 1 41 1...   
9  /000/147            1  for 2 jeff 2 36 2 33 1 mailer 1 38 1 exchange ...   

   spam_word_count  
0                0  
1                0  
2                0  
3                0  


('Accuracy score:', 58.058058058058059)
('R^2:', -0.722413793103448)
('Mean squared error:', 0.4194194194194194)
Root mean squared error: 0.647625987295
('Mean absolute error:', 0.4194194194194194)